# To search Danbooru tags by most tagged with franchise:
https://danbooru.donmai.us/related_tag.json?query={$SEARCH_TERM}&category=character&is_deprecated=false&order=count

[Reference](https://danbooru.donmai.us/wiki_pages/help:api)

In [1]:
import json
import urllib.request
from urllib.request import urlopen
from tqdm.auto import tqdm
import time

In [2]:
character = "angela_(seiken_densetsu_3)"
limit = 1000

In [3]:
def myurlopen(url, max_wait=300):
    i = 2
    while True:
        try:
            req = urllib.request.Request(
                url,
                data=None,
                headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'}
            )
            r = urlopen(req)
            return r
        except:
            sleep_time = 2**i / 1000
            if sleep_time > max_wait:
                print('ABORT!')
                raise TimeoutError(f'Timed out with max wait time of {max_wait} s')
            time.sleep(sleep_time)
            i += 1
            continue

In [88]:
template_text = """
'$CHARACTER, $THE_FRANCHISE, $TAGS':
  Description:
    - $CHARACTER upsample tags
  Tags:
    - Characters
    - $FRANCHISE
    - $CHARACTER
"""

In [ ]:
final_text = ''

tags = []
tag_overlap = 0.25

url = f"https://danbooru.donmai.us/related_tag.json?query={character}&category=general&is_deprecated=false&order=count&limit={limit}"
response = myurlopen(url)
related_tags = json.loads(response.read())['related_tags']

for rtag in tqdm(related_tags):
    if rtag['overlap_coefficient'] <= tag_overlap:
        continue
    tag_dict = rtag['tag']
    if tag_dict['post_count'] < 100:
        continue

    if tag_dict['name'].endswith('eyes') or tag_dict['name'].endswith('hair') or ('skin' in tag_dict['name']) or ('tail' in tag_dict['name']) or tag_dict['name'].endswith('ears') or ('ahoge' in tag_dict['name']):

        tags.append( tag_dict['name'] )
    # tags.append( tag_dict['name'] )

tags = set(tags)
processed = [ x.strip().replace('_', ' ').replace('(', '\(').replace(')', '\)') for x in tags]
tag_string = ', '.join(processed)
final_text += template_text.replace('$TAGS', tag_string).replace("$CHARACTER", character)

  0%|          | 0/500 [00:00<?, ?it/s]

In [90]:
tag_string

'green eyes, pointy ears, very long hair, long hair, blonde hair, purple hair'